In [2]:
# TODO:
%load_ext autoreload
%autoreload 2
import torch
import torch.onnx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# matplotlib.use('Qt5Agg')
%matplotlib Qt5Agg
import livenet
from livenet import nets, datasets


In [3]:
downscale = (14, 14)
train_x, train_y = datasets.to_plain(*datasets.get_mnist_train(), downscale=downscale, to_odd=True, to_gray=True)



In [11]:

network = nets.create_perceptron(train_x.shape[1], 2, 2)
res = network(train_x)
network.context.regularization_l1 = 0.001
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
#optimizer = livenet.backend.optimizers.optimizers.LiveNetOptimizer(network, lr=0.01)
optimizer = nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=50)
trainer.clear_life_stat = False



LiveNet


In [12]:
network.mortal = False
trainer.step(250)
network.mortal = True
trainer.step(250)



Iˈ0.000 49 1.000+0.002reg params=16 av_abs_grad 0.003 av_abs_delta: 0.006 av_abs_output 0.03 max_abs_output 0.05 max_abs_param 0.7 hcut 0 lcut 0.4 livenet/net_trainer.py:122
Iˈ0.204 99 0.995+0.001reg params=16 av_abs_grad 0.006 av_abs_delta: 0.003 av_abs_output 0.04 max_abs_output 0.05 max_abs_param 0.4 hcut 0 lcut 0.2 livenet/net_trainer.py:122
Iˈ0.412 149 0.967+0.002reg params=16 av_abs_grad 0.01 av_abs_delta: 0.003 av_abs_output 0.1 max_abs_output 0.1 max_abs_param 0.9 hcut 0 lcut 0.3 livenet/net_trainer.py:122
Iˈ0.617 199 0.923+0.003reg params=16 av_abs_grad 0.02 av_abs_delta: 0.003 av_abs_output 0.1 max_abs_output 0.2 max_abs_param 1.4 hcut 0 lcut 0.3 livenet/net_trainer.py:122
Iˈ0.828 249 0.874+0.005reg params=16 av_abs_grad 0.02 av_abs_delta: 0.003 av_abs_output 0.2 max_abs_output 0.3 max_abs_param 1.9 hcut 0 lcut 0.3 livenet/net_trainer.py:122
Iˈ0.832 N1 became useless and will die at tick 250 livenet/backend/core.py:83
Iˈ0.832 killing DestinationNeuron N1 with b=-0.001, tick=2

In [8]:

pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y.numpy() - pred_bin
accuracy = len(diff[diff == 0]) / len(diff)
accuracy


0.7581166666666667

In [34]:

trainer.step(1)


In [6]:
#livenet.utils.export_onnx(network, "/home/spometun/table/home/model.onnx")
print(f"total: {len(list(network.named_parameters()))}")
print(network.context.topology_stat.get_stat())
def name_comp_key(x: str):
    if "->" in x:
        x = "_" + x
    return x

for name, p in sorted(network.named_parameters(), key=lambda x: name_comp_key(x[0])):
    print(name, f"{p.item():.2f}")


total: 16
{'dangle': {'RegularNeuron': 0, 'DestinationNeuron': 0}, 'useless': {'RegularNeuron': 0, 'InputNeuron': 0}}
D0 -0.02
D1 -0.02
N0 0.00
N1 0.00
I0->N0 0.26
I0->N1 0.31
I1->N0 0.12
I1->N1 0.40
I2->N0 -0.25
I2->N1 -0.19
I3->N0 0.41
I3->N1 0.23
N0->D0 -0.31
N0->D1 0.56
N1->D0 0.68
N1->D1 0.26


In [7]:
life_stat = network.context.life_stat
df = pd.DataFrame(life_stat)
# df2 = df[(df["type"] == "parameter") & (df["name"] == "I0->N0")]
df2 = df[(df["type"] == "output_low_cut_ratio") & (df["name"] == "N0")]
df2.set_index("tick")
df2["value"].plot(grid=True, x="tick", y="value", style='.-', use_index=False)


<Axes: >

In [11]:
values = df2["value"].to_list()
ticks = list(df2["tick"])

plt.ion()
plt.figure(figsize=(16, 9))
plt.grid()
plt.plot(y=values, x=ticks)
# plt.show()

TypeError: plot() got an unexpected keyword argument 'x'

In [16]:
print(df2["tick"].head())

61     0
119    1
177    2
235    3
293    4
Name: tick, dtype: int64
